TensorFlow 与 Keras 之间存在既竞争，又合作的交错关系，甚至连 Keras 创始人都在Google 工作。早在 2015 年 11 月，TensorFlow 就被加入 Keras 后端支持。从 2017 年开始，Keras 的大部分组件被整合到 TensorFlow 框架中。2019 年，在 TensorFlow 2 版本中，Keras被正式确定为 TensorFlow 的高层唯一接口 API，取代了 TensorFlow 1 版本中自带的tf.layers 等高层接口。也就是说，现在只能使用 Keras 的接口来完成 TensorFlow 层方式的模型搭建与训练。在 TensorFlow 中，Keras 被实现在 tf.keras 子模块中。

## 常见功能模块
Keras 提供了一系列高层的神经网络相关类和函数，如经典数据集加载函数、网络层类、模型容器、损失函数类、优化器类、经典模型类等

### 常见的网络层类
对于常见的神经网络层，可以使用张量方式的底层接口函数来实现，这些接口函数一般在 tf.nn 模块中。更常用地，对于常见的网络层，我们一般直接使用**层方式**来完成模型的搭建，在 **tf.keras.layers** 命名空间(下文使用 layers 指代 tf.keras.layers)中提供了大量常见网络层的类，如全连接层、激活函数层、池化层、卷积层、循环神经网络层等。对于这些网络层类，只需要在创建时指定网络层的相关参数，并调用_call_方法即可完成前向计算。在
调用_call_方法时，Keras 会自动调用每个层的前向传播逻辑，这些逻辑一般实现在类的call 函数中。  
以 Softmax 层为例，它既可以使用 tf.nn.softmax 函数在前向传播逻辑中完成 Softmax运算，也可以通过 layers.Softmax(axis)类搭建 Softmax 网络层，其中 axis 参数指定进行softmax 运算的维度。

In [1]:
import tensorflow as tf

In [2]:
x = tf.constant([2.,1.,0.1]) # 创建输入张量
layer = tf.keras.layers.Softmax(axis=-1) # 创建 Softmax 层
out = layer(x) 
out

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.6590012 , 0.24243298, 0.09856589], dtype=float32)>

### 网络容器
对于常见的网络，需要手动调用每一层的类实例完成前向传播运算，当网络层数变得较深时，这一部分代码显得非常臃肿。**可以通过 Keras 提供的网络容器 Sequential 将多个网络层封装成一个大网络模型，只需要调用网络模型的实例一次即可完成数据从第一层到最末层的顺序传播运算**。

In [3]:
network = tf.keras.Sequential([ 
 tf.keras.layers.Dense(3, activation=None), 
 tf.keras.layers.ReLU(),
 tf.keras.layers.Dense(2, activation=None), 
 tf.keras.layers.ReLU() 
])
x = tf.random.normal([4,3])
out = network(x)
network.summary()
out

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (4, 3)                    12        
_________________________________________________________________
re_lu (ReLU)                 (4, 3)                    0         
_________________________________________________________________
dense_1 (Dense)              (4, 2)                    8         
_________________________________________________________________
re_lu_1 (ReLU)               (4, 2)                    0         
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[4.7105393e-01, 2.0222282e-02],
       [1.0642592e+00, 6.4952083e-02],
       [0.0000000e+00, 0.0000000e+00],
       [1.1055711e-01, 5.5796240e-04]], dtype=float32)>

Sequential 容器也可以通过 add()方法继续追加新的网络层，实现动态创建网络的功能.

In [4]:
layers_num = 2 
network = tf.keras.Sequential([]) 
for _ in range(layers_num):
    network.add(tf.keras.layers.Dense(3)) 
    network.add(tf.keras.layers.ReLU())
network.build(input_shape=(4, 4))
network.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (4, 3)                    15        
_________________________________________________________________
re_lu_2 (ReLU)               (4, 3)                    0         
_________________________________________________________________
dense_3 (Dense)              (4, 3)                    12        
_________________________________________________________________
re_lu_3 (ReLU)               (4, 3)                    0         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________


当我们通过 Sequential 容量封装多个网络层时，每层的参数列表将会自动并入Sequential 容器的参数列表中，不需要人为合并网络参数列表，这也是 Sequential 容器的便捷之处。Sequential 对象的 trainable_variables 和 variables 包含了所有层的待优化张量列表和全部张量列表.

In [5]:
for p in network.trainable_variables:
    print(p.name, p.shape)

dense_2/kernel:0 (4, 3)
dense_2/bias:0 (3,)
dense_3/kernel:0 (3, 3)
dense_3/bias:0 (3,)


## 模型装配、训练与测试
在训练网络时，一般的流程是通过前向计算获得网络的输出值，再通过损失函数计算网络误差，然后通过自动求导工具计算梯度并更新，同时间隔性地测试网络的性能。对于这种常用的训练逻辑，可以直接通过 Keras 提供的模型装配与训练等高层接口实现，简洁清晰。

### 模型装配
在 Keras 中，有 2 个比较特殊的类：keras.Model 和 keras.layers.Layer 类。其中 Layer类是网络层的母类，定义了网络层的一些常见功能，如添加权值、管理权值列表等。Model 类是网络的母类，除了具有 Layer 类的功能，还添加了保存模型、加载模型、训练与测试模型等便捷功能。Sequential 也是 Model 的子类，因此具有 Model 类的所有功能。

In [6]:
(x, y), (x_val, y_val) = tf.keras.datasets.mnist.load_data()
x = 2*tf.convert_to_tensor(x, dtype=tf.float32) / 255.-1  
x=tf.reshape(x,[-1,28*28])
y = tf.convert_to_tensor(y, dtype=tf.int32)
y = tf.one_hot(y, depth=10) 
x_val = 2*tf.convert_to_tensor(x_val, dtype=tf.float32) / 255.-1 
x_val=tf.reshape(x_val,[-1,28*28])
y_val= tf.convert_to_tensor(y_val, dtype=tf.int32)
y_val = tf.one_hot(y_val, depth=10) 
train_db = tf.data.Dataset.from_tensor_slices((x, y)).batch(32)
val_db=tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(32)

In [7]:
network = tf.keras.Sequential([
 tf.keras.layers.Dense(256, activation='relu'),
 tf.keras.layers.Dense(128, activation='relu'),
 tf.keras.layers.Dense(64, activation='relu'),
 tf.keras.layers.Dense(32, activation='relu'),
 tf.keras.layers.Dense(10)])
network.build(input_shape=(1,28*28))
network.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (1, 256)                  200960    
_________________________________________________________________
dense_5 (Dense)              (1, 128)                  32896     
_________________________________________________________________
dense_6 (Dense)              (1, 64)                   8256      
_________________________________________________________________
dense_7 (Dense)              (1, 32)                   2080      
_________________________________________________________________
dense_8 (Dense)              (1, 10)                   330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________


In [8]:
network.compile(optimizer='adam',
loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
metrics=['accuracy']
)

### 模型训练
模型装配完成后，即可通过 fit()函数送入待训练的数据集和验证用的数据集，这一步称为模型训练.

In [9]:
history = network.fit(train_db, epochs=5, validation_data=val_db, validation_freq=2)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3268 - accuracy: 0.8993
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1710 - accuracy: 0.9472 - val_loss: 0.2241 - val_accuracy: 0.9293
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1346 - accuracy: 0.9591
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1128 - accuracy: 0.9657 - val_loss: 0.1754 - val_accuracy: 0.9475
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0961 - accuracy: 0.9707


In [10]:
history.history

{'loss': [0.3268168866634369,
  0.17100563645362854,
  0.13455285131931305,
  0.11277803778648376,
  0.09605088084936142],
 'accuracy': [0.8992666602134705,
  0.9471666812896729,
  0.9590666890144348,
  0.9656500220298767,
  0.970716655254364],
 'val_loss': [0.22411838173866272, 0.17542347311973572],
 'val_accuracy': [0.9293000102043152, 0.9474999904632568]}

### 模型测试
Model 基类除了可以便捷地完成网络的装配与训练、验证，还可以非常方便的预测和测试。

In [11]:
out = network.predict(x)
out.shape,tf.argmax(out,axis=1)[0:10],tf.argmax(y,axis=1)[0:10]

((60000, 10),
 <tf.Tensor: shape=(10,), dtype=int64, numpy=array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4], dtype=int64)>,
 <tf.Tensor: shape=(10,), dtype=int64, numpy=array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4], dtype=int64)>)

In [12]:
network.evaluate(val_db)

313/313 [==============================] - 0s 1ms/step - loss: 0.1397 - accuracy: 0.9600


[0.13973765075206757, 0.9599999785423279]

## 模型加载和保存
模型训练完成后，需要将模型保存到文件系统上，从而方便后续的模型测试与部署工作。实际上，在训练时间隔性地保存模型状态也是非常好的习惯，这一点对于训练大规模的网络尤其重要。**一般大规模的网络需要训练数天乃至数周的时长，一旦训练过程被中断或者发生宕机等意外，之前训练的进度将全部丢失。如果能够间断地保存模型状态到文件系统，即使发生宕机等意外，也可以从最近一次的网络状态文件中恢复，从而避免浪费大量的训练时间和计算资源**。因此模型的保存与加载非常重要。  
在 Keras 中，有三种常用的模型保存与加载方法。

### 张量方式
网络的状态主要体现在网络的结构以及网络层内部张量数据上，因此在拥有网络结构源文件的条件下，直接保存网络张量参数到文件系统上是最轻量级的一种方式。通过调用 Model.save_weights(path)方法即可将当前的网络参数保存到 path 文件上.  

In [13]:
network.save_weights('./Save_weight/weights.ckpt')
print('saved weights.')

saved weights.


In [14]:
del network # 删除网络对象

In [15]:
# 重新创建相同的网络结构
network = tf.keras.Sequential([tf.keras.layers.Dense(256, activation='relu'),
 tf.keras.layers.Dense(128, activation='relu'),
 tf.keras.layers.Dense(64, activation='relu'),
 tf.keras.layers.Dense(32, activation='relu'),
 tf.keras.layers.Dense(10)])
network.compile(optimizer='adam',
loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
metrics=['accuracy']
 ) 
# 从参数文件中读取数据并写入当前网络
network.load_weights('./Save_weight/weights.ckpt')
print('loaded weights!')

loaded weights!


In [16]:
network.compile(optimizer='adam',
loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
metrics=['accuracy']
)
history = network.fit(train_db, epochs=5, validation_data=val_db, validation_freq=2)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0856 - accuracy: 0.9734
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0772 - accuracy: 0.9764 - val_loss: 0.1305 - val_accuracy: 0.9660
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0712 - accuracy: 0.9777
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0641 - accuracy: 0.9799 - val_loss: 0.1376 - val_accuracy: 0.9652
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0609 - accuracy: 0.9811


这种保存与加载网络的方式最为轻量级，文件中保存的仅仅是张量参数的数值，并没有其它额外的结构参数。但是它需要使用相同的网络结构才能够正确恢复网络状态，因此一般在拥有网络源文件的情况下使用。

### 网络方式

我们来介绍一种不需要网络源文件，仅仅需要模型参数文件即可恢复出网络模型的方法。通过 Model.save(path)函数可以将模型的结构以及模型的参数保存到 path 文件上，在不需要网络源文件的条件下，通过 keras.models.load_model(path)即可恢复网络结构和网络参数。

In [17]:
network.save('./Save_weight/model.h5')
print('saved total model.')

saved total model.


In [18]:
del network # 删除网络对象

In [19]:
# 从文件恢复网络结构与网络参数
network = tf.keras.models.load_model('./Save_weight/model.h5')

In [20]:
network.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 256)               200960    
_________________________________________________________________
dense_10 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_11 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_12 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_13 (Dense)             (None, 10)                330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________


In [21]:
network.compile(optimizer='adam',
loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
metrics=['accuracy']
)
history = network.fit(train_db, epochs=5, validation_data=val_db, validation_freq=2)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0568 - accuracy: 0.9823
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0517 - accuracy: 0.9839 - val_loss: 0.1123 - val_accuracy: 0.9726
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0486 - accuracy: 0.9848
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0459 - accuracy: 0.9856 - val_loss: 0.1411 - val_accuracy: 0.9672
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0453 - accuracy: 0.9855


In [22]:
for p in network.trainable_variables:
    print(p.name, p.shape)

dense_9/kernel:0 (784, 256)
dense_9/bias:0 (256,)
dense_10/kernel:0 (256, 128)
dense_10/bias:0 (128,)
dense_11/kernel:0 (128, 64)
dense_11/bias:0 (64,)
dense_12/kernel:0 (64, 32)
dense_12/bias:0 (32,)
dense_13/kernel:0 (32, 10)
dense_13/bias:0 (10,)


### SavedModel方式
TensorFlow 之所以能够被业界青睐，除了优秀的神经网络层 API 支持之外，还得益于它强大的生态系统，包括移动端和网页端等的支持。当需要将模型部署到其他平台时，采用 TensorFlow 提出的 SavedModel 方式更具有平台无关性。  
通过 tf.saved_model.save (network, path)即可将模型以 SavedModel 方式保存到 path 目录中.

In [23]:
tf.saved_model.save(network, './Save_weight/SaveModel')
print('saving savedmodel.')


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.
INFO:tensorflow:Assets written to: ./Save_weight/SaveModel\assets
saving savedmodel.


用户无需关心文件的保存格式，只需要通过 tf.saved_model.load 函数即可恢复出模型对象，我们在恢复出模型实例后，完成测试准确率的计算

In [24]:
del network

In [25]:
print('load savedmodel from file.')
# 从文件恢复网络结构与网络参数
network = tf.saved_model.load('./Save_weight/SaveModel')

load savedmodel from file.


In [26]:
acc_meter = tf.metrics.CategoricalAccuracy()
acc_meter.reset_states()
for x,y in val_db: 
    pred = network(x) 
    acc_meter.update_state(tf.argmax(y,axis=1), tf.argmax(pred,axis=1)) # 更新准确率统计
# 打印准确率
print("Test Accuracy:%f" % acc_meter.result())

Test Accuracy:0.936102


## 自定义网络
尽管 Keras 提供了很多的常用网络层类，但深度学习可以使用的网络层远远不止这些。科研工作者一般是自行实现了较为新颖的网络层，经过大量实验验证有效后，深度学习框架才会跟进，内置对这些网络层的支持。因此掌握自定义网络层、网络的实现非常重要。  

对于需要创建自定义逻辑的网络层，可以通过自定义类来实现。在创建自定义网络层类时，需要继承自 layers.Layer 基类；创建自定义的网络类时，需要继承自 keras.Model 基类，这样建立的自定义类才能够方便的利用 Layer/Model 基类提供的参数管理等功能，同时也能够与其他的标准网络层类交互使用。

### 自定义网络层
对于自定义的网络层，至少需要实现初始化__init__方法和前向传播逻辑 call 方法。我们以某个具体的自定义网络层为例，假设需要一个没有偏置向量的全连接层，即 bias 为0，同时固定激活函数为 ReLU 函数。

In [27]:
class MyDense(tf.keras.layers.Layer):
    def __init__(self,in_dim,out_dim):
        super(MyDense,self).__init__()
        self.kernel=self.add_weight('w',[in_dim,out_dim],trainable=True)
    def call(self, inputs, training=None):
        out = inputs @ self.kernel
        out = tf.nn.relu(out)
        return out
net = MyDense(4,3)
net.variables,net.trainable_variables

([<tf.Variable 'w:0' shape=(4, 3) dtype=float32, numpy=
  array([[-0.03383017, -0.7945089 , -0.25250584],
         [-0.19947702,  0.40103364,  0.31436467],
         [ 0.85367143, -0.47576442,  0.8969743 ],
         [ 0.27099824,  0.41371226,  0.3759085 ]], dtype=float32)>],
 [<tf.Variable 'w:0' shape=(4, 3) dtype=float32, numpy=
  array([[-0.03383017, -0.7945089 , -0.25250584],
         [-0.19947702,  0.40103364,  0.31436467],
         [ 0.85367143, -0.47576442,  0.8969743 ],
         [ 0.27099824,  0.41371226,  0.3759085 ]], dtype=float32)>])

通过修改为 self.kernel = self.add_variable('w', [inp_dim, outp_dim], trainable=False)，我们可以设置𝑾张量不需要被优化.


自定义类的前向运算逻辑实现在 call(inputs, training=None)函数中，其中 inputs代表输入，由用户在调用时传入；training 参数用于指定模型的状态：training 为 True 时执行训练模式，training 为 False 时执行测试模式，默认参数为 None，即测试模式。由于全连接层的训练模式和测试模式逻辑一致，此处不需要额外处理。对于部份测试模式和训练模式不一致的网络层，需要根据 training 参数来设计需要执行的逻辑。

In [28]:
network = tf.keras.Sequential([MyDense(784, 256), # 使用自定义的层
 MyDense(256, 128),
 MyDense(128, 64),
 MyDense(64, 32),
 MyDense(32, 10)])
network.build(input_shape=(None, 28*28))
network.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_dense_1 (MyDense)         (None, 256)               200704    
_________________________________________________________________
my_dense_2 (MyDense)         (None, 128)               32768     
_________________________________________________________________
my_dense_3 (MyDense)         (None, 64)                8192      
_________________________________________________________________
my_dense_4 (MyDense)         (None, 32)                2048      
_________________________________________________________________
my_dense_5 (MyDense)         (None, 10)                320       
Total params: 244,032
Trainable params: 244,032
Non-trainable params: 0
_________________________________________________________________


### 自定义网络

In [29]:
class MyModel(tf.keras.Model):
# 自定义网络类，继承自 Model 基类
    def __init__(self):
        super(MyModel, self).__init__()
        # 完成网络内需要的网络层的创建工作
        self.fc1 = MyDense(28*28, 256)
        self.fc2 = MyDense(256, 128)
        self.fc3 = MyDense(128, 64)
        self.fc4 = MyDense(64, 32)
        self.fc5 = MyDense(32, 10)
    def call(self, inputs, training=None):
        # 自定义前向运算逻辑
        x = self.fc1(inputs) 
        x = self.fc2(x) 
        x = self.fc3(x) 
        x = self.fc4(x) 
        x = self.fc5(x) 
        return x

In [30]:
model = MyModel()

In [31]:
model.build(input_shape=(None,28*28))

In [32]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_dense_6 (MyDense)         multiple                  200704    
_________________________________________________________________
my_dense_7 (MyDense)         multiple                  32768     
_________________________________________________________________
my_dense_8 (MyDense)         multiple                  8192      
_________________________________________________________________
my_dense_9 (MyDense)         multiple                  2048      
_________________________________________________________________
my_dense_10 (MyDense)        multiple                  320       
Total params: 244,032
Trainable params: 244,032
Non-trainable params: 0
_________________________________________________________________


## 模型乐园
对于常用的网络模型，如 ResNet、VGG 等，不需要手动创建网络，可以直接从keras.applications 子模块中通过一行代码即可创建并使用这些经典模型，同时还可以通过设置 weights 参数加载预训练的网络参数，非常方便。

以 ResNet50 网络模型为例，一般将 ResNet50 去除最后一层后的网络作为新任务的特征提取子网络，即利用在 ImageNet 数据集上预训练好的网络参数初始化，并根据自定义任务的类别追加一个对应数据类别数的全连接分类层或子网络，从而可以在预训练网络的基础上快速、高效地学习新任务。

In [33]:
# 加载 ImageNet 预训练网络模型，并去掉最后一层
resnet = tf.keras.applications.ResNet50(weights='imagenet',include_top=False)
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [34]:
# 测试网络的输出
x = tf.random.normal([4,224,224,3])
out = resnet(x) # 获得子网络的输出
out.shape

TensorShape([4, 7, 7, 2048])

以 100 类的分类任务为例，我们在 ResNet50 基础上重新构建新网络。新建一个池化层(这里的池化层暂时可以理解为高、宽维度下采样的功能)，将特征从[𝑏, 7,7,2048]降维到[𝑏, 2048]

In [35]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
# 利用上一层的输出作为本层的输入，测试其输出
x = tf.random.normal([4,7,7,2048])
# 池化层降维，形状由[4,7,7,2048]变为[4,1,1,2048],删减维度后变为[4,2048]
out = global_average_layer(x)
print(out.shape)

(4, 2048)


In [36]:
# 新建全连接层
fc = tf.keras.layers.Dense(100)
# 利用上一层的输出[4,2048]作为本层的输入，测试其输出
x = tf.random.normal([4,2048])
out = fc(x) # 输出层的输出为样本属于 100 类别的概率分布
print(out.shape)

(4, 100)


In [37]:
mynet = tf.keras.Sequential([resnet, global_average_layer, fc])
mynet.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, None, None, 2048)  23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 100)               204900    
Total params: 23,792,612
Trainable params: 23,739,492
Non-trainable params: 53,120
_________________________________________________________________


通过设置 resnet.trainable = False 可以选择冻结 ResNet 部分的网络参数，只训练新建的网络层，从而快速、高效完成网络模型的训练。当然也可以在自定义任务上更新网络的全部参数。

## 测量工具

在网络的训练过程中，经常需要统计准确率、召回率等测量指标，除了可以通过手动计算的方式获取这些统计数据外，Keras 提供了一些常用的测量工具，位于 keras.metrics 模块中，专门用于统计训练过程中常用的指标数据。Keras 的测量工具的使用方法一般有 4 个主要步骤：新建测量器，写入数据，读取统计数据和清零测量器.

### 新建测量器
在 keras.metrics 模块中，提供了较多的常用测量器类，如统计平均值的 Mean 类，统计准确率的 Accuracy 类，统计余弦相似度的 CosineSimilarity 类等。

In [38]:
loss_meter = tf.keras.metrics.Mean()

### 写入数据
通过测量器的 update_state 函数可以写入新的数据，测量器会根据自身逻辑记录并处理采样数据。

In [ ]:
loss_meter.update_state(float(loss))

### 读取统计信息
在采样多次数据后，可以选择在需要的地方调用测量器的 result()函数，来获取统计值。

In [ ]:
print(step, 'loss:', loss_meter.result()) 

### 清除状态
由于测量器会统计所有历史记录的数据，因此在启动新一轮统计时，有必要清除历史状态。通过 reset_states()即可实现清除状态功能。

In [ ]:
if step % 100 == 0:
    # 打印统计的平均 loss
    print(step, 'loss:', loss_meter.result()) 
    loss_meter.reset_states() # 打印完后，清零测量器